In [ ]:
#@title Import statements {display-mode: "form"}
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
from newspaper import Article
from bs4 import BeautifulSoup
import urllib3
import nltk
# from nltk import pos_tag
# from nltk.corpus import stopwords, words
# from nltk.stem import WordNetLemmatizer
# from nltk.corpus import wordnet
nltk.download('stopwords')
nltk.download('words')
nltk.download('movie_reviews')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
import string
import re
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load("en_core_web_sm")
# import textacy
# from textacy import preprocessing
import pandas as pd
import numpy as np

In [ ]:
#@title Load special libraries {display-mode: "form"}
!pip install -q newspaper3k
!pip install -q textblob
!pip install spacy
!conda install -c conda-forge spacy
!python -m spacy download en
!pip install textacy
!conda install -c conda-forge textacy
!python -m textacy download
!pip install spacy-lookups-data

In [ ]:
class MyArticle:
  def __init__(self,url):
    self.url=url
    self.article=Article(url)
    self.article.download()
    self.article.parse()
    self.title = self.article.title
    self.text = self.article.text
    self.tokenized = self.dataCleaning()
    self.sentiment = TextBlob(self.text,analyzer=NaiveBayesAnalyzer())
    self.polarity = self.sentiment.polarity
    self.subjectivity = self.sentiment.subjectivity
    self.score = self.score()

  def dataCleaning(self):
    text=self.text.lower()
    tokenized=nlp(text)
    tokenized=[word for word in tokenized if word.is_stop is False]
    tokenized=[word for word in tokenized if word.is_alpha is True]
    tokenized=' '.join([word.lemma_ for word in tokenized])
    return tokenized
  
  def score(self):
    if self.sentiment.polarity >= 0.05: 
      return 'positive'
    elif -0.05 < self.sentiment.polarity < 0.05:
      return  'neutral'
    else:
      return 'negative'

# Web Scraping

In [ ]:
def getURLs(base):
	http = urllib3.PoolManager()
	response = http.request('GET', base)
 
	tempLinks = re.findall(r'/.*?html', response.data.decode("utf-8"))
	links = []
	for tempLink in tempLinks:
		if not re.search(r'(/topics/|w3|/html|/programmes/|<|>|https:|/aboutus/|/watch_now/)',tempLink) and tempLink:
			# print(tempLink)
			links.append(tempLink)

	return list(dict.fromkeys(["https://www.aljazeera.com"+x for x in links])) 

In [ ]:
urls = getURLs("https://www.aljazeera.com/")

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [ ]:
print(urls)
articles=[]

for url in urls:
     articles.append(MyArticle(url))

['https://www.aljazeera.com/news/2020/06/eu-bar-travellers-brazil-coronavirus-live-updates-200627002953075.html', 'https://www.aljazeera.com/news/2020/06/governors-slow-reopening-states-covid-19-cases-surge-200626160805422.html', 'https://www.aljazeera.com/news/2020/06/covid-19-prison-problem-cases-soar-california-san-quentin-200625184510342.html', 'https://www.aljazeera.com/news/2020/06/coronavirus-crisis-exposes-india-social-inequalities-200626060045684.html', 'https://www.aljazeera.com/news/2020/06/ethiopia-agrees-delay-filling-nile-mega-dam-egypt-sudan-200627025116180.html', 'https://www.aljazeera.com/news/2020/06/ethiopia-fill-nile-dam-july-agreement-reached-200626085142188.html', 'https://www.aljazeera.com/news/2020/06/sudan-millions-risk-ethiopia-fills-mega-dam-deal-200625150231487.html', 'https://www.aljazeera.com/news/2020/06/challenges-set-extend-dangerous-mission-200619163508338.html', 'https://www.aljazeera.com/profile/joseph-stepansky.html', 'https://www.aljazeera.com/news

In [ ]:
def output(index): #Accepts sentiment result (double string), url (string), plain text article (str)

  title = df.iloc[index,0]
  text = df.iloc[index,1]
  url = df.iloc[index,2]
  result = str(df.iloc[index,3])
  print(title+"\n"+url+"\n"+"Sentiment Analysis result: "+result+"\n\n\n"+text)


In [ ]:
data={
    'Title':[i.title for i in articles],
    'Text':[i.text for i in articles],
    'Link':[i.url for i in articles],
    'Polarity':[i.polarity for i in articles],
    'Subjectivity':[i.subjectivity for i in articles],
    'Label':[i.score for i in articles],
    }
df=pd.DataFrame(data)

In [ ]:
df

,Title,Text,Link,Polarity,Subjectivity,Label
0,"EU to bar travellers from US, Brazil: Coronavi...",The European Union plans to bar travellers fro...,https://www.aljazeera.com/news/2020/06/eu-bar-...,0.082557,0.421084,positive
1,US governors slow reopening states as COVID-19...,When Texas began lifting coronavirus restricti...,https://www.aljazeera.com/news/2020/06/governo...,0.101324,0.398895,positive
2,COVID-19 prison problem as cases soar at Calif...,The California state jail system has seen a st...,https://www.aljazeera.com/news/2020/06/covid-1...,0.105532,0.427906,positive
3,How the coronavirus crisis exposes India's soc...,When Pradeep Kumar's wife was admitted to a go...,https://www.aljazeera.com/news/2020/06/coronav...,0.093775,0.426774,positive
4,Ethiopia agrees to delay filling Nile mega-dam...,"The leaders of Sudan, Ethiopia and Egypt agree...",https://www.aljazeera.com/news/2020/06/ethiopi...,0.031552,0.456193,neutral
5,Ethiopia to fill Nile dam in July even if 'no ...,Ethiopia has reiterated it will start filling ...,https://www.aljazeera.com/news/2020/06/ethiopi...,0.088750,0.418333,positive
6,Sudan: Millions at risk if Ethiopia fills mega...,Sudan has sent a letter to the United Nations ...,https://www.aljazeera.com/news/2020/06/sudan-m...,0.041867,0.390411,neutral
7,Challenges ahead as UN set to extend 'most dan...,The United Nations Security Council is expecte...,https://www.aljazeera.com/news/2020/06/challen...,0.048319,0.345101,neutral
8,Joseph Stepansky,Joseph Stepansky\n\nJoseph Stepansky is freela...,https://www.aljazeera.com/profile/joseph-stepa...,0.088699,0.311869,positive
9,US imposes new sanctions on Chinese leaders ov...,Secretary of State Mike Pompeo said on Friday ...,https://www.aljazeera.com/news/2020/06/imposes...,0.024539,0.295691,neutral


In [ ]:
while True:
  output(int(input("Enter article number \n>> ")))

Yannick Giovanni Marshall
https://www.aljazeera.com/profile/yannick-marshall.html
Sentiment Analysis result: -0.16666666666666666


United States 01 Jun 2020 17:33 GMT

Black liberal, your time is up

Yes, tell the world that we are fed up. But, Black liberal, know that we are finished with you, too.
